# ГЛАВА 7

В этом ноутбуке начинает разбираться тема создание модели с несколькими входами и несколькими выходами

# Создание модели привычным способом и с помощью API

In [ ]:
# keras API позволяет напрямую манипулировать тензорами и 
# рассматривать слои как функции
from keras.models import Sequential, Model
from keras.layers import Dense
from keras import Input
import numpy as np

# Привычный способ
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(64,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Эквивалент через API
# создаем входной тензор
input_tensor = Input(shape=(64,))
# в первый слой передаем входной тензор
x = Dense(128, activation='relu')(input_tensor)
# во второй слой передаем первый слой
x = Dense(64, activation='relu')(x)
# в каждый последующий слой передаем предыдущий
output_tensor = Dense(10, activation='softmax')(x)

# создаем модель, указывая входной и выходной тензоры
model = Model(input_tensor, output_tensor)

model.summary()

# компиляция, обучение, оценка - такие же
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['mae']
)

x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=20
)

score = model.evaluate(x_train, y_train)


# Создание модели типа "вопрос-ответ" с несколькими ВХОДАМИ

In [ ]:
# обычно модель "вопрос-ответ" получает на вход вопрос на естественном языке
# и текст, на основе которого будет даваться ответ

from keras.models import Model
from keras import layers
from keras import Input

text_size = 10_000
question_size = 10_000
answer_size = 500

# входной текст - последовательность целых чисел переменной длины
# форма - (количество образцов, длина образца)
text_input = Input(shape=(None,), dtype='int32', name='text')
# преобразование текста в последовательность векторов размером 64
embedded_text = layers.Embedding(text_size, 64)(text_input)
# преобразование в единый вектор
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='float32', name='question')
embedded_question = layers.Embedding(question_size, 64)(question_input)
encoded_question = layers.LSTM(32)(embedded_question)

# объединение закодированного вопроса и ответа
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# создание классификатора
answer = layers.Dense(answer_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# ПЕРЕДАЧА ДАННЫХ В МОДЕЛЬ С НЕСКОЛЬКИМИ ВХОДАМИ
num_samples = 1000
max_length = 100

text = np.random.randint(1, text_size, size=(num_samples, max_length))
question = np.random.randint(1, question_size, size=(num_samples, max_length))
answers = np.zeros(shape=(num_samples, answer_size))
# массив индексов
indices = np.random.randint(0, answer_size, size=num_samples)
# прямое кодирование вопросов
for i, x in enumerate(answers):
  x[indices[i]] = 1

# можно передать так
# то есть после тренировки answers будет заполнено значениями
model.fit([text, question], answers, epochs=20, batch_size=128)
# но если при создании через Input тензорам давались имена, то можно передать и через словарь
model.fit({'text':text, 'question':question}, answers, epochs=20, batch_size=128)

model.evaluate([text, question], answers)

Epoch 1/20
8/8 [==============================] - 0s 26ms/step - loss: 6.2145 - accuracy: 0.0020
Epoch 2/20
8/8 [==============================] - 0s 26ms/step - loss: 6.1890 - accuracy: 0.0330
Epoch 3/20
8/8 [==============================] - 0s 25ms/step - loss: 6.1026 - accuracy: 0.0040
Epoch 4/20
8/8 [==============================] - 0s 25ms/step - loss: 6.0131 - accuracy: 0.0030
Epoch 5/20
8/8 [==============================] - 0s 26ms/step - loss: 5.9218 - accuracy: 0.0050
Epoch 6/20
8/8 [==============================] - 0s 25ms/step - loss: 5.8282 - accuracy: 0.0130
Epoch 7/20
8/8 [==============================] - 0s 27ms/step - loss: 5.7401 - accuracy: 0.0230
Epoch 8/20
8/8 [==============================] - 0s 25ms/step - loss: 5.6365 - accuracy: 0.0270
Epoch 9/20
8/8 [==============================] - 0s 24ms/step - loss: 5.5529 - accuracy: 0.0370
Epoch 10/20
8/8 [==============================] - 0s 25ms/step - loss: 5.4571 - accuracy: 0.0290
Epoch 11/20
8/8 [============

[3.438575267791748, 0.5130000114440918]

# Создание модели с тремя ВЫХОДАМИ

In [ ]:
# иногда модель может принимать один набор данных и по нему предсказывать разные 
# характеристики
# например на вход идет пост из соцсети, а на выходе - возраст, пол, зароботок человека
from keras import layers 
from keras import Input
from keras.models import Model

vocabulary_size = 50_000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='float32')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling2D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# для выходных слоев лучше определить имена
# СКАЛЯРНАЯ РЕГРЕССИЯ
age_prediction = layers.Dense(1, name='age')(x)
# этот слой выдает вероятность принадлежности к опредленной группе - поэтому функция softmax
# СКАЛЯРНАЯ РЕГРЕССИЯ
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
# это слой выдает один из двух вариантов - мужчина или женщина - поэтомму функция sigmoid
# БИНАРНАЯ КЛАССИФИКАЦИЯ
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

# из-за того, что здесь сразу несколько задач, при компиляции модели необходимо 
# указывать список разных функций потерь, которые затем будут суммироваться в один 
# финальный тензор для вычисления градиета
# однако обычно функия mse колеблется у значения 3-5, а перекрестная энтропия - 
# у значения 0.1
# общая функция потерь будет более приспособлена к той функции, значения которой БОЛЬШЕ
# то есть для задачи с большей потерей
# чтобы это исправить можно ввести ограничения на то, какой силы эффект оказывает каждая 
# из функций потерь на общую функцию, получаемую из их суммы
model.compile(
    optimizer='rmsprop',
    loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
    loss_weights = [0.25, 1., 10.]
)
# так запускается тренировка (у меня просто нет данных для ее начала)
# то есть модель обработает posts и заполнит результатами члены массива
# изначально члены массива - пустые значения, которые НАДО СОЗДАТЬ
# model.fit(posts, [predicted_age, predicted_income, predicted_gender], epochs=100, batch_size = 64)